# Crime Scene Investigation

## Fighting un-Pythonic Crime

### ByteCode At a Time

#### Let [PEP-8](https://www.python.org/dev/peps/pep-0008/) Be With You!

My tool of choice is **dis** - Python dissasembly module

In [1]:
import dis
dis.__doc__

'Disassembler of Python byte code into mnemonics.'

In [2]:
dis.dis('x = 1')

  1           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (x)
              4 LOAD_CONST               1 (None)
              6 RETURN_VALUE


I created a helper function for side-by-side presentation _(not seen in demo mode)_

It makes the "crime" more obvious

**Pay attention to bytecode length**

In [3]:
import dis
import io
import itertools
import re
import sys

class IoCatcher:
    def __init__(self):
        self.old_stdout = sys.stdout
        sys.stdout = self.buffer = io.StringIO()
    def __enter__(self):
        return self.buffer
    def __exit__(self, *_):
        sys.stdout = self.old_stdout

def comparative_dis(*snippets):
    compressor = re.compile(r'( +)'
                             '(  | \d|\d\d)' # 2
                             '( +)'
                             '(  |>>)' # 4
                             '( +)'
                             '([ \d]\d +\w+)' #6
                             '( {4})?'
                           )    
    snippets_dis = []
    for snippet in snippets:
        with IoCatcher() as io_catcher:
            dis.dis(snippet)
        snippets_dis.append(list(map(lambda l: compressor.sub(r' \2 \4 \6', l),
                                     io_catcher.getvalue().splitlines())))
    widths = [max(map(len, snippet)) for snippet in snippets_dis]
    fmt = ' | '.join('{{:<{}}}'.format(w) for w in widths)
    print(*(fmt.format(*lines) for lines in itertools.zip_longest(*snippets_dis, fillvalue='')), 
          sep='\n')

## Truthiness of Sequences

> For sequences, (strings, lists, tuples), use the fact that empty sequences are false:

```python
# Correct:
if not seq:
if seq:

# Wrong:
if len(seq):
if not len(seq):

```

### The Proof

In [4]:
comparative_dis('if seq: pass', 'if len(seq): pass')

  1     0 LOAD_NAME            0 (seq)  |   1     0 LOAD_NAME            0 (len) 
        2 POP_JUMP_IF_FALSE    4        |         2 LOAD_NAME            1 (seq) 
    >>  4 LOAD_CONST           0 (None) |         4 CALL_FUNCTION        1       
        6 RETURN_VALUE                  |         6 POP_JUMP_IF_FALSE    8       
                                        |     >>  8 LOAD_CONST           0 (None)
                                        |        10 RETURN_VALUE                 


In [5]:
comparative_dis('if not seq: pass', 'if not len(seq): pass')

  1     0 LOAD_NAME            0 (seq)  |   1     0 LOAD_NAME            0 (len) 
        2 POP_JUMP_IF_TRUE     4        |         2 LOAD_NAME            1 (seq) 
    >>  4 LOAD_CONST           0 (None) |         4 CALL_FUNCTION        1       
        6 RETURN_VALUE                  |         6 POP_JUMP_IF_TRUE     8       
                                        |     >>  8 LOAD_CONST           0 (None)
                                        |        10 RETURN_VALUE                 


## Booleans - again

> Don't compare boolean values to _True_ or _False_ using __==__:

```python
# Correct:
if greeting:

# Wrong:
if greeting == True:

# Worse:
if greeting is True:
```

### The Proof

In [6]:
comparative_dis('if flag: pass', 'if flag == True: pass', 'if flag is True: pass')

  1     0 LOAD_NAME            0 (flag) |   1     0 LOAD_NAME            0 (flag) |   1     0 LOAD_NAME            0 (flag)
        2 POP_JUMP_IF_FALSE    4        |         2 LOAD_CONST           0 (True) |         2 LOAD_CONST           0 (True)
    >>  4 LOAD_CONST           0 (None) |         4 COMPARE_OP           2 (==)   |         4 IS_OP                0       
        6 RETURN_VALUE                  |         6 POP_JUMP_IF_FALSE    8        |         6 POP_JUMP_IF_FALSE    8       
                                        |     >>  8 LOAD_CONST           1 (None) |     >>  8 LOAD_CONST           1 (None)
                                        |        10 RETURN_VALUE                  |        10 RETURN_VALUE                 


## Type Check
> Object type comparisons should always use isinstance() instead of comparing types directly:

```python
# Correct:
if isinstance(obj, int):

# Wrong:
if type(obj) is int:

# Worst
if type(obj) == int:
```

### The Proof

In [7]:
comparative_dis('if isinstance(obj, int): pass', 'if type(obj) is int: pass', 'if type(obj) == int: pass')

  1     0 LOAD_NAME            0 (isinstance) |   1     0 LOAD_NAME            0 (type) |   1     0 LOAD_NAME            0 (type)
        2 LOAD_NAME            1 (obj)        |         2 LOAD_NAME            1 (obj)  |         2 LOAD_NAME            1 (obj) 
        4 LOAD_NAME            2 (int)        |         4 CALL_FUNCTION        1        |         4 CALL_FUNCTION        1       
        6 CALL_FUNCTION        2              |         6 LOAD_NAME            2 (int)  |         6 LOAD_NAME            2 (int) 
        8 POP_JUMP_IF_FALSE   10              |         8 IS_OP                0        |         8 COMPARE_OP           2 (==)  
    >> 10 LOAD_CONST           0 (None)       |        10 POP_JUMP_IF_FALSE   12        |        10 POP_JUMP_IF_FALSE   12       
       12 RETURN_VALUE                        |     >> 12 LOAD_CONST           0 (None) |     >> 12 LOAD_CONST           0 (None)
                                              |        14 RETURN_VALUE                  | 

# So far, PEP-8 is partially covered. What next?

## Key in a Dictionary

```python
# Correct 
if 'key' in some_dict:
for key in some_dict:
    
# Wrong
if 'key' in some_dict.keys()
for key in some_dict.keys():
```


### The Proof

In [8]:
comparative_dis('"key" in some_dict', '"key" in some_dict.keys()')

  1     0 LOAD_CONST           0 ('key')     |   1     0 LOAD_CONST           0 ('key')    
        2 LOAD_NAME            0 (some_dict) |         2 LOAD_NAME            0 (some_dict)
        4 CONTAINS_OP          0             |         4 LOAD_METHOD          1 (keys)     
        6 RETURN_VALUE                       |         6 CALL_METHOD          0            
                                             |         8 CONTAINS_OP          0            
                                             |        10 RETURN_VALUE                      


In [9]:
comparative_dis('for key in some_dict: pass', 'for key in some_dict.keys(): pass')

  1     0 LOAD_NAME            0 (some_dict) |   1     0 LOAD_NAME            0 (some_dict)
        2 GET_ITER                           |         2 LOAD_METHOD          1 (keys)     
    >>  4 FOR_ITER             4 (to 10)     |         4 CALL_METHOD          0            
        6 STORE_NAME           1 (key)       |         6 GET_ITER                          
        8 JUMP_ABSOLUTE        4             |     >>  8 FOR_ITER             4 (to 14)    
    >> 10 LOAD_CONST           0 (None)      |        10 STORE_NAME           2 (key)      
       12 RETURN_VALUE                       |        12 JUMP_ABSOLUTE        8            
                                             |     >> 14 LOAD_CONST           0 (None)     
                                             |        16 RETURN_VALUE                      


## Iteration Over List

In [10]:
def sum_list_correct(arr):
    list_sum = 0
    for elem in arr:
        list_sum += elem
    return list_sum

def sum_list_incorrect(arr):
    list_sum = 0
    for idx in range(len(arr)):
        list_sum += arr[idx]
    return list_sum

### The Proof

In [11]:
comparative_dis(sum_list_correct, sum_list_incorrect)

  2     0 LOAD_CONST           1 (0)        |   8     0 LOAD_CONST           1 (0)       
        2 STORE_FAST           1 (list_sum) |         2 STORE_FAST           1 (list_sum)
                                            |                                            
  3     4 LOAD_FAST            0 (arr)      |   9     4 LOAD_GLOBAL          0 (range)   
        6 GET_ITER                          |         6 LOAD_GLOBAL          1 (len)     
    >>  8 FOR_ITER            12 (to 22)    |         8 LOAD_FAST            0 (arr)     
       10 STORE_FAST           2 (elem)     |        10 CALL_FUNCTION        1           
                                            |        12 CALL_FUNCTION        1           
  4    12 LOAD_FAST            1 (list_sum) |        14 GET_ITER                         
       14 LOAD_FAST            2 (elem)     |     >> 16 FOR_ITER            16 (to 34)   
       16 INPLACE_ADD                       |        18 STORE_FAST           2 (idx)     
       18 

### Set Abuse
Ever hear about set literal?

In [12]:
comparative_dis('set([1, 2, 3])', '{1, 2, 3}')

  1     0 LOAD_NAME            0 (set)       |   1     0 BUILD_SET            0                       
        2 BUILD_LIST           0             |         2 LOAD_CONST           0 (frozenset({1, 2, 3}))
        4 LOAD_CONST           0 ((1, 2, 3)) |         4 SET_UPDATE           1                       
        6 LIST_EXTEND          1             |         6 RETURN_VALUE                                 
        8 CALL_FUNCTION        1             |                                                        
       10 RETURN_VALUE                       |                                                        
